In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import easydict
from torchsummary import summary

# argument parser
import easydict

args = easydict.EasyDict({
        "batch_size": 32,
        "epochs": 10,
        "lr": 0.01,
})
# Hyper Parameters
input_size = 784
num_classes = 10
num_epochs = args.epochs
batch_size = args.batch_size
learning_rate = args.lr

# MNIST Dataset (Images and Labels)
train_set = dsets.FashionMNIST(
    root = './data/FashionMNIST',
    train = True,
    download = True,
    transform = transforms.Compose([
        transforms.ToTensor()
    ])
)
test_set = dsets.FashionMNIST(
    root = './data/FashionMNIST',
    train = False,
    download = True,
    transform = transforms.Compose([
        transforms.ToTensor()
    ])
)


# Dataset Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset = train_set,
        batch_size = batch_size,
        shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_set,
        batch_size = batch_size,
        shuffle = False)

100%|██████████| 26.4M/26.4M [00:02<00:00, 12.7MB/s]


Extracting ./data/FashionMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 202kB/s]


Extracting ./data/FashionMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:01<00:00, 3.76MB/s]


Extracting ./data/FashionMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 17.7MB/s]

Extracting ./data/FashionMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/FashionMNIST/raw



In [ ]:
class MyConvNet(nn.Module):
    def __init__(self, args):
        super(MyConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.act1  = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.act2  = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.lin2  = nn.Linear(7*7*32, 10, bias=False)

    def forward(self, x):
        x = self.conv1(x)
        x = self.act1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.act2(x)
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = self.lin2(x)
        return x

model = MyConvNet(args)
model = model.cuda()

criterion = nn.CrossEntropyLoss()
criterion=criterion.cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 5e-4, momentum=0.9)

In [ ]:
summary(model, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             144
              ReLU-2           [-1, 16, 28, 28]               0
         MaxPool2d-3           [-1, 16, 14, 14]               0
            Conv2d-4           [-1, 32, 14, 14]           4,608
              ReLU-5           [-1, 32, 14, 14]               0
         MaxPool2d-6             [-1, 32, 7, 7]               0
            Linear-7                   [-1, 10]          15,680
Total params: 20,432
Trainable params: 20,432
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.32
Params size (MB): 0.08
Estimated Total Size (MB): 0.40
----------------------------------------------------------------


In [ ]:
print("---Training started")
# Training the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.cuda()
        labels = Variable(labels).cuda()

        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        L1norm = model.parameters()
        arr = []
        for name,param in model.named_parameters():
          if 'weight' in name.split('.'):
            arr.append(param)
        L1loss = 0
        for Losstmp in arr:
          L1loss = L1loss+Losstmp.abs().mean()

        loss.backward()
        optimizer.step()

        if (i + 1) % 600 == 0:
            print('Epoch: [% d/% d], Step: [% d/% d], Loss: %.4f'
                    % (epoch + 1, num_epochs, i + 1,
                       len(train_set) // batch_size, loss.data.item()))

correct = 0
total = 0
model.eval()
for images, labels in test_loader:
    images = images.cuda()
    labels = labels.cuda()
    outputs = model(images)
    testloss = criterion(outputs, labels)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy for test images: % d %%' % (100 * correct / total))

---Training started
Epoch: [ 1/ 10], Step: [ 600/ 1875], Loss: 0.3892
Epoch: [ 1/ 10], Step: [ 1200/ 1875], Loss: 0.1307
Epoch: [ 1/ 10], Step: [ 1800/ 1875], Loss: 0.6903
Epoch: [ 2/ 10], Step: [ 600/ 1875], Loss: 0.3467
Epoch: [ 2/ 10], Step: [ 1200/ 1875], Loss: 0.2349
Epoch: [ 2/ 10], Step: [ 1800/ 1875], Loss: 0.3979
Epoch: [ 3/ 10], Step: [ 600/ 1875], Loss: 0.1799
Epoch: [ 3/ 10], Step: [ 1200/ 1875], Loss: 0.3809
Epoch: [ 3/ 10], Step: [ 1800/ 1875], Loss: 0.4327
Epoch: [ 4/ 10], Step: [ 600/ 1875], Loss: 0.1758
Epoch: [ 4/ 10], Step: [ 1200/ 1875], Loss: 0.4870
Epoch: [ 4/ 10], Step: [ 1800/ 1875], Loss: 0.2944
Epoch: [ 5/ 10], Step: [ 600/ 1875], Loss: 0.2735
Epoch: [ 5/ 10], Step: [ 1200/ 1875], Loss: 0.4666
Epoch: [ 5/ 10], Step: [ 1800/ 1875], Loss: 0.2832
Epoch: [ 6/ 10], Step: [ 600/ 1875], Loss: 0.1872
Epoch: [ 6/ 10], Step: [ 1200/ 1875], Loss: 0.1457
Epoch: [ 6/ 10], Step: [ 1800/ 1875], Loss: 0.1563
Epoch: [ 7/ 10], Step: [ 600/ 1875], Loss: 0.0951
Epoch: [ 7/ 10], S

In [ ]:
torch.save(model.state_dict(), './saved_myconvnet_lab4.pt')

In [ ]:
load_model = MyConvNet(args)
load_model.load_state_dict(torch.load('./saved_myconvnet_lab4.pt', weights_only=False))

load_model = load_model.cuda()
correct = 0
total = 0
load_model.eval()
for images, labels in test_loader:
    images = images.cuda()
    labels = labels.cuda()
    outputs = load_model(images)
    testloss = criterion(outputs, labels)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy for test images: % d %%' % (100 * correct / total))

Accuracy for test images:  90 %


In [ ]:
### Lab 3 Part 1 -
### Quantization functions
from copy import deepcopy
import torch.nn.utils.prune as prune

def stochastic_round(x):
    x_low = torch.floor(x)
    x_high = torch.ceil(x)
    frac = x - x_low

    return torch.where(torch.rand_like(x) < frac, x_high, x_low)

def squant(max_value, num_bits, x):
    # Uniform symetric quantizer that
    # quantize x into sf * qx
    # sf: scaling factor
    # qx: integer in range [-2^(bits-1)+1, 2^(bits-1)-1]
    # note: only 2^bits - 1 different values can be represented, bits >= 2

    sf = max_value / (2**(num_bits-1) - 1)
    qx = stochastic_round(x/sf)
    qx = torch.clip(qx,min=-2**(num_bits-1) + 1, max=2**(num_bits-1) - 1)
    dqx = qx * sf
    return dqx


def asquant(max_value, min_value, num_bits, x):
    # Uniform Asymetric quantizer that
    # quantize x into min_value + sf * qx
    # sf: scaling factor
    # qx: integer in range [0,2^bits-1]
    # note: 2^bits different values can be represented, bits >= 1

    sf = (max_value - min_value) / (2**num_bits - 1)
    qx = stochastic_round((x-min_value)/sf)
    qx = torch.clip(qx,min=0,max=2**(num_bits) - 1)
    dqx = min_value + qx * sf
    return dqx

### General Functions

def evaluate_model(test_model, model_name):
    print("-------------------------------------")
    print(f"Evaluating {model_name}")

    correct = 0
    total = 0
    test_model.eval()
    for images, labels in test_loader:
        images = images.cuda()
        labels = labels.cuda()
        outputs = test_model(images)
        testloss = criterion(outputs, labels)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Accuracy on the 10000 test images: % .2f %%' % (100 * correct/ total))
    print()
    print("Size of", model_name)
    print(f"Total parameters: {sum(p.numel() for p in test_model.parameters())}")
    print()

    all_non_zeros = 0

    print("Parameters per layer:")
    for name, param in test_model.named_parameters():
        if 'weight' in name:
            non_zero = param.nonzero().size(0)
            all_non_zeros += non_zero
            total_params = param.numel()
            print(f"Layer: {name} | Non-zero params: {non_zero} | Total params: {total_params} | Sparsity: {100 * (1 -  non_zero / total_params):.2f}%")

    print(f"Model has {all_non_zeros} non-zero params in total")
    print()

    param_size = 4  # 4 bytes per float32 parameter
    model_size_kb = all_non_zeros * param_size / 1024
    print(f"Approximate model size: {model_size_kb:.2f} KB")
    print("-------------------------------------")


In [ ]:
sd = load_model.state_dict() # state dictionary of original model

In [ ]:
### Signed Symmetrical Quantization
q_model = deepcopy(load_model)
q_sd = q_model.state_dict()

max_val = 0
for name,_ in load_model.named_parameters():
    if torch.max(torch.abs(sd[name])) > max_val:
        max_val = torch.max(torch.abs(sd[name]))

# Sweep over different bitwidths
for n_bits in (4, 8, 12, 16):
    print(f'quantizing model into {n_bits} bits')
    for name,_ in load_model.named_parameters():
        # print('quantizing ',name)
        q_sd[name] = squant(max_val, n_bits, sd[name])

    q_model.load_state_dict(q_sd)

    evaluate_model(q_model, f"Symm Quant Model, n={n_bits}")

quantizing model into 4 bits
Accuracy on the 10000 test images:  82.35 %

quantizing model into 8 bits
Accuracy on the 10000 test images:  89.58 %

quantizing model into 12 bits
Accuracy on the 10000 test images:  89.63 %

quantizing model into 16 bits
Accuracy on the 10000 test images:  89.61 %



In [ ]:
### Asymmetrical Quantization
q_model = deepcopy(load_model)
q_sd = q_model.state_dict()

max_val = 0
min_val = 1000
for name,_ in load_model.named_parameters():
    if torch.max(sd[name]) > max_val:
        max_val = torch.max(sd[name])
    if torch.min(sd[name]) < min_val:
        min_val = torch.min(sd[name])

# Sweep over different bitwidths
for n_bits in (4, 8, 12, 16):
    print(f'quantizing model into {n_bits} bits')
    for name,_ in load_model.named_parameters():
        # print('quantizing ',name)
        q_sd[name] = asquant(max_val, min_val, n_bits, sd[name])

    q_model.load_state_dict(q_sd)

    evaluate_model(q_model, f"Asymm Quant Model, n={n_bits}")

quantizing model into 4 bits
Accuracy on the 10000 test images:  87.18 %

quantizing model into 8 bits
Accuracy on the 10000 test images:  89.67 %

quantizing model into 12 bits
Accuracy on the 10000 test images:  89.61 %

quantizing model into 16 bits
Accuracy on the 10000 test images:  89.61 %



In [ ]:
### For Signed Symmetrical Quantization - but this time we will change [q_min, q_max]
q_model = deepcopy(load_model)
q_sd = q_model.state_dict()

max_vals = []
for name,_ in load_model.named_parameters():
    max_vals.append(torch.max(torch.abs(sd[name])))
max_vals.sort(reverse=True)

# Sweep over different bitwidths
print("Limiting range for signed symmetrical quantization:")
for n_bits in (4, 8, 12, 16):
    for limit in range(len(max_vals)): # for taking the nth highest and lowest values in tensor
        if limit == 0:
            continue

        print(f'quantizing model into {n_bits} bits, and limiting range by {limit} values')

        for name,_ in load_model.named_parameters():
            # print('quantizing ',name)
            q_sd[name] = squant(max_vals[limit], n_bits, sd[name])

        q_model.load_state_dict(q_sd)

        evaluate_model(q_model, f"Range limited Symm Quant Model, n={n_bits}, lim={limit}")

Limiting range for signed symmetrical quantization:
quantizing model into 4 bits, and limiting range by 1 values
Accuracy on the 10000 test images:  88.21 %
-------------------------------------
Size of Range limited Symmetrically quantized model bit_width=4, n=1
Total parameters: 20432
Parameters per layer:
Layer: conv1.weight | Non-zero params: 132 | Total params: 144 | Sparsity: 8.33%
Layer: conv2.weight | Non-zero params: 2884 | Total params: 4608 | Sparsity: 37.41%
Layer: lin2.weight | Non-zero params: 8244 | Total params: 15680 | Sparsity: 47.42%
Model has 11260 non-zero params in total

Approximate model size: 43.98 KB
-------------------------------------
quantizing model into 4 bits, and limiting range by 2 values
Accuracy on the 10000 test images:  87.45 %
-------------------------------------
Size of Range limited Symmetrically quantized model bit_width=4, n=2
Total parameters: 20432
Parameters per layer:
Layer: conv1.weight | Non-zero params: 135 | Total params: 144 | Spars

In [ ]:
### Same for Asymmetrical Quantization
q_model_asymm_2 = deepcopy(load_model)
q_asymm_sd_2 = q_model_asymm_2.state_dict()

max_vals = []
min_vals = []
for name,_ in load_model.named_parameters():
    max_vals.append(torch.max(sd[name]))
    min_vals.append(torch.min(sd[name]))
max_vals.sort(reverse=True)
min_vals.sort()

print("Limiting range for asymmetrical quantization:")
for n_bits in (4, 8, 12, 16):
    for limit in range(len(max_vals)): # for taking the nth highest and lowest values in tensor
        if limit == 0:
            continue

        print(f'quantizing model into {n_bits} bits, and limiting range by {limit} values')

        for name,_ in load_model.named_parameters():
            # print('quantizing ',name)
            q_asymm_sd_2[name] = asquant(max_vals[limit], min_vals[limit], n_bits, sd[name])

        q_model_asymm_2.load_state_dict(q_asymm_sd_2)

        # Test the Model
        evaluate_model(q_model, f"Range limited Asymm Quant Model, n={n_bits}, lim={limit}")

Limiting range for asymmetrical quantization:
quantizing model into 4 bits, and limiting range by 1 values
Accuracy on the 10000 test images:  86.96 %
-------------------------------------
Size of Range limited Asymmetrically quantized model bit_width=4, n=1
Total parameters: 20432
Parameters per layer:
Layer: conv1.weight | Non-zero params: 144 | Total params: 144 | Sparsity: 0.00%
Layer: conv2.weight | Non-zero params: 4608 | Total params: 4608 | Sparsity: 0.00%
Layer: lin2.weight | Non-zero params: 15680 | Total params: 15680 | Sparsity: 0.00%
Model has 20432 non-zero params in total

Approximate model size: 79.81 KB
-------------------------------------
quantizing model into 4 bits, and limiting range by 2 values
Accuracy on the 10000 test images:  87.06 %
-------------------------------------
Size of Range limited Asymmetrically quantized model bit_width=4, n=2
Total parameters: 20432
Parameters per layer:
Layer: conv1.weight | Non-zero params: 144 | Total params: 144 | Sparsity: 

In [ ]:
### For per-layer Symmetrical Quantization
q_model_symm_3 = deepcopy(load_model)
q_symm_sd_3 = q_model_symm_3.state_dict()

# Sweep over different bitwidths
for n_bits in (4, 8, 12, 16):
    print(f'quantizing model into {n_bits} bits')
    for name,_ in load_model.named_parameters():
        # print('quantizing ',name)
        q_symm_sd_3[name] = squant(torch.max(torch.abs(sd[name])), n_bits, sd[name])

    q_model_symm_3.load_state_dict(q_symm_sd_3)

    # Test the Model
    evaluate_model(q_model_symm_3, f"Symmetrically quantized model, per-layer with bit_width = {n_bits}")

quantizing model into 4 bits
Accuracy on the 10000 test images:  89.15 %
-------------------------------------
Size of Symmetrically quantized model, per-layer with bit_width = 4
Total parameters: 20432
Parameters per layer:
Layer: conv1.weight | Non-zero params: 111 | Total params: 144 | Sparsity: 22.92%
Layer: conv2.weight | Non-zero params: 2928 | Total params: 4608 | Sparsity: 36.46%
Layer: lin2.weight | Non-zero params: 8799 | Total params: 15680 | Sparsity: 43.88%
Model has 11838 non-zero params in total

Approximate model size: 46.24 KB
-------------------------------------
quantizing model into 8 bits
Accuracy on the 10000 test images:  90.19 %
-------------------------------------
Size of Symmetrically quantized model, per-layer with bit_width = 8
Total parameters: 20432
Parameters per layer:
Layer: conv1.weight | Non-zero params: 140 | Total params: 144 | Sparsity: 2.78%
Layer: conv2.weight | Non-zero params: 4487 | Total params: 4608 | Sparsity: 2.63%
Layer: lin2.weight | No

In [ ]:
### For per-layer Asymmetrical Quantization
q_model_asymm_3 = deepcopy(load_model)
q_asymm_sd_3 = q_model_asymm_3.state_dict()

# Sweep over different bitwidths
for n_bits in (4, 8, 12, 16):
    print(f'quantizing model into {n_bits} bits')
    for name,_ in load_model.named_parameters():
        # print('quantizing ',name)
        q_asymm_sd_3[name] = asquant(torch.max(sd[name]), torch.min(sd[name]), n_bits, sd[name])

    q_model_asymm_3.load_state_dict(q_asymm_sd_3)

    evaluate_model(q_model_asymm_3, f"Asymmetrically quantized model, per-layer with bit_width = {n_bits}")

quantizing model into 4 bits
Accuracy on the 10000 test images:  89.38 %
-------------------------------------
Size of Asymmetrically quantized model, per-layer with bit_width = 4
Total parameters: 20432
Parameters per layer:
Layer: conv1.weight | Non-zero params: 144 | Total params: 144 | Sparsity: 0.00%
Layer: conv2.weight | Non-zero params: 4608 | Total params: 4608 | Sparsity: 0.00%
Layer: lin2.weight | Non-zero params: 15680 | Total params: 15680 | Sparsity: 0.00%
Model has 20432 non-zero params in total

Approximate model size: 79.81 KB
-------------------------------------
quantizing model into 8 bits
Accuracy on the 10000 test images:  90.22 %
-------------------------------------
Size of Asymmetrically quantized model, per-layer with bit_width = 8
Total parameters: 20432
Parameters per layer:
Layer: conv1.weight | Non-zero params: 144 | Total params: 144 | Sparsity: 0.00%
Layer: conv2.weight | Non-zero params: 4608 | Total params: 4608 | Sparsity: 0.00%
Layer: lin2.weight | No

In [ ]:
### Task 2: Activation + Weight Quantization

max_val = 0
min_val = 1000
for name,_ in model.named_parameters():
    if torch.max(sd[name]) > max_val:
        max_val = torch.max(sd[name])
    if torch.min(sd[name]) < min_val:
        min_val = torch.min(sd[name])

class MyConvNet2(nn.Module):
    def __init__(self, args):
        super(MyConvNet2, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.act1  = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.act2  = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.lin2  = nn.Linear(7*7*32, 10, bias=False)

    def forward(self, x):
        x = asquant(max_val, min_val, 16, self.conv1(x))
        x = asquant(max_val, min_val, 16, self.act1(x))
        x = asquant(max_val, min_val, 16, self.pool1(x))
        x = asquant(max_val, min_val, 16, self.conv2(x))
        x = asquant(max_val, min_val, 16, self.act2(x))
        x = asquant(max_val, min_val, 16, self.pool2(x))
        x = x.view(x.size(0), -1)
        x = self.lin2(x)
        return x

model = MyConvNet2(args)
model = model.cuda()

criterion = nn.CrossEntropyLoss()
criterion=criterion.cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 5e-4, momentum=0.9)

print("---Training started for Task 2: Activation + Weight Quantization")
# Training the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.cuda()
        labels = Variable(labels).cuda()

        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        L1norm = model.parameters()
        arr = []
        for name,param in model.named_parameters():
          if 'weight' in name.split('.'):
            arr.append(param)
        L1loss = 0
        for Losstmp in arr:
          L1loss = L1loss+Losstmp.abs().mean()

        loss.backward()
        optimizer.step()

        if (i + 1) % 600 == 0:
            print('Epoch: [% d/% d], Step: [% d/% d], Loss: %.4f'
                    % (epoch + 1, num_epochs, i + 1,
                       len(train_set) // batch_size, loss.data.item()))

correct = 0
total = 0
model.eval()
for images, labels in test_loader:
    images = images.cuda()
    labels = labels.cuda()
    outputs = model(images)
    testloss = criterion(outputs, labels)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy for test images: % d %%' % (100 * correct / total))


---Training started for Task 2: Activation + Weight Quantization
Epoch: [ 1/ 10], Step: [ 600/ 1875], Loss: 0.7032
Epoch: [ 1/ 10], Step: [ 1200/ 1875], Loss: 0.7046
Epoch: [ 1/ 10], Step: [ 1800/ 1875], Loss: 0.7637
Epoch: [ 2/ 10], Step: [ 600/ 1875], Loss: 0.4903
Epoch: [ 2/ 10], Step: [ 1200/ 1875], Loss: 0.5681
Epoch: [ 2/ 10], Step: [ 1800/ 1875], Loss: 0.6625
Epoch: [ 3/ 10], Step: [ 600/ 1875], Loss: 0.5949
Epoch: [ 3/ 10], Step: [ 1200/ 1875], Loss: 0.6217
Epoch: [ 3/ 10], Step: [ 1800/ 1875], Loss: 0.6995
Epoch: [ 4/ 10], Step: [ 600/ 1875], Loss: 0.6880
Epoch: [ 4/ 10], Step: [ 1200/ 1875], Loss: 0.7280
Epoch: [ 4/ 10], Step: [ 1800/ 1875], Loss: 0.6007
Epoch: [ 5/ 10], Step: [ 600/ 1875], Loss: 0.7549
Epoch: [ 5/ 10], Step: [ 1200/ 1875], Loss: 0.8557
Epoch: [ 5/ 10], Step: [ 1800/ 1875], Loss: 0.9410
Epoch: [ 6/ 10], Step: [ 600/ 1875], Loss: 0.7295
Epoch: [ 6/ 10], Step: [ 1200/ 1875], Loss: 0.8787
Epoch: [ 6/ 10], Step: [ 1800/ 1875], Loss: 0.8188
Epoch: [ 7/ 10], Step: 

In [ ]:
### Pruning 1

criterion = nn.CrossEntropyLoss()
criterion = criterion.cuda()
prune_amounts = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5]

print("Filter-wise pruning based on L1 norm")
for amount in prune_amounts:

    pruned_model = deepcopy(load_model)

    for name, module in pruned_model.named_modules():
        if isinstance(module, nn.Conv2d):
            prune.ln_structured(module, name='weight', amount=amount, n=1, dim=0)
            # prune.ln_structured(module, name='weight', amount=amount, n=2, dim=0)
            prune.remove(module, 'weight')

    evaluate_model(pruned_model, f"Filter-wise pruning based on L1 norm, amount={amount}")

Filter-wise pruning based on L1 norm
-------------------------------------
Evaluating Filter-wise pruning based on L1 norm, amount=0.05
Accuracy on the 10000 test images:  90.25 %

Size of Filter-wise pruning based on L1 norm, amount=0.05
Total parameters: 20432

Parameters per layer:
Layer: conv1.weight | Non-zero params: 135 | Total params: 144 | Sparsity: 6.25%
Layer: conv2.weight | Non-zero params: 4320 | Total params: 4608 | Sparsity: 6.25%
Layer: lin2.weight | Non-zero params: 15680 | Total params: 15680 | Sparsity: 0.00%
Model has 20135 non-zero params in total

Approximate model size: 78.65 KB
-------------------------------------
-------------------------------------
Evaluating Filter-wise pruning based on L1 norm, amount=0.1
Accuracy on the 10000 test images:  90.19 %

Size of Filter-wise pruning based on L1 norm, amount=0.1
Total parameters: 20432

Parameters per layer:
Layer: conv1.weight | Non-zero params: 126 | Total params: 144 | Sparsity: 12.50%
Layer: conv2.weight | No

In [ ]:
### Pruning 2

criterion = nn.CrossEntropyLoss()
criterion = criterion.cuda()
prune_amounts = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5]

print("Channel-wise pruning based on L1 and L2 norm")
for amount in prune_amounts:

    pruned_model = deepcopy(load_model)

    for name, module in pruned_model.named_modules():
        if isinstance(module, nn.Conv2d):
            prune.ln_structured(module, name='weight', amount=amount, n=1, dim=1)
            prune.ln_structured(module, name='weight', amount=amount, n=2, dim=1)
            prune.remove(module, 'weight')

    evaluate_model(pruned_model, f"Channel-wise pruning based on L1 and L2 norm, amount={amount}")


Channel-wise pruning based on L1 and L2 norm
-------------------------------------
Evaluating Channel-wise pruning based on L1 and L2 norm, amount=0.05
Accuracy on the 10000 test images:  90.20 %

Size of Channel-wise pruning based on L1 and L2 norm, amount=0.05
Total parameters: 20432

Parameters per layer:
Layer: conv1.weight | Non-zero params: 144 | Total params: 144 | Sparsity: 0.00%
Layer: conv2.weight | Non-zero params: 4032 | Total params: 4608 | Sparsity: 12.50%
Layer: lin2.weight | Non-zero params: 15680 | Total params: 15680 | Sparsity: 0.00%
Model has 19856 non-zero params in total

Approximate model size: 77.56 KB
-------------------------------------
-------------------------------------
Evaluating Channel-wise pruning based on L1 and L2 norm, amount=0.1
Accuracy on the 10000 test images:  90.09 %

Size of Channel-wise pruning based on L1 and L2 norm, amount=0.1
Total parameters: 20432

Parameters per layer:
Layer: conv1.weight | Non-zero params: 144 | Total params: 144 | 

In [ ]:
### Optimized pruning
prune_amounts = [0.1, 0.2, 0.22, 0.3]
evaluate_model(load_model, "Original Model")

print("Combine both Channel and filter -wise pruning")
for amount in prune_amounts:

    pruned_model = deepcopy(load_model)

    for name, module in pruned_model.named_modules():
        if isinstance(module, nn.Conv2d):
            prune.ln_structured(module, name='weight', amount=amount, n=1, dim=0)
            prune.ln_structured(module, name='weight', amount=amount, n=1, dim=1)
            prune.ln_structured(module, name='weight', amount=amount, n=2, dim=1)
            prune.remove(module, 'weight')

    evaluate_model(pruned_model, "Pruned Model")


-------------------------------------
Size of Original Model
Total parameters: 20432
Parameters per layer:
Layer: conv1.weight | Non-zero params: 144 | Total params: 144 | Sparsity: 0.00%
Layer: conv2.weight | Non-zero params: 4608 | Total params: 4608 | Sparsity: 0.00%
Layer: lin2.weight | Non-zero params: 15680 | Total params: 15680 | Sparsity: 0.00%
Model has 20432 non-zero params in total

Approximate model size: 79.81 KB
-------------------------------------
Combine both Channel and filter -wise pruning
With prune amount of 0.1 Accuracy for test images:  90 %
-------------------------------------
Size of Pruned Model
Total parameters: 20432
Parameters per layer:
Layer: conv1.weight | Non-zero params: 126 | Total params: 144 | Sparsity: 12.50%
Layer: conv2.weight | Non-zero params: 3393 | Total params: 4608 | Sparsity: 26.37%
Layer: lin2.weight | Non-zero params: 15680 | Total params: 15680 | Sparsity: 0.00%
Model has 19199 non-zero params in total

Approximate model size: 75.00 KB

In [ ]:
### Combine both quantization and pruning to achieve an 80% accurate model
### that is as small as possible

quantized_model = deepcopy(load_model)
new_sd = load_model.state_dict()

# Quantize the model
n_bits = 4
print(f'Perform per-layer quantization with bit width of {n_bits} bits')
for name,_ in load_model.named_parameters():
    # print('quantizing ',name)
    new_sd[name] = squant(torch.max(torch.abs(sd[name])), n_bits, sd[name])

quantized_model.load_state_dict(new_sd)

prune_amounts = [0.3, 0.4, 0.5, 0.6]

# REALLY aggressive pruning
for amount in prune_amounts:
    print(f'Prune the model with amount = {amount * 100} % :')
    pruned_model = deepcopy(quantized_model)
    for name, module in pruned_model.named_modules():
        if isinstance(module, nn.Conv2d):
            # prune.ln_structured(module, name='weight', amount=amount, n=1, dim=0)
            prune.ln_structured(module, name='weight', amount=amount, n=1, dim=1)
            prune.ln_structured(module, name='weight', amount=amount, n=2, dim=1)
            prune.remove(module, 'weight')

    evaluate_model(pruned_model, "80% accurate Model")

Perform per-layer quantization with bit width of 4 bits
Prune the model with amount = 30.0 % :
-------------------------------------
Evaluating 80% accurate Model
Accuracy on the 10000 test images:  82.55 %

Size of 80% accurate Model
Total parameters: 20432

Parameters per layer:
Layer: conv1.weight | Non-zero params: 114 | Total params: 144 | Sparsity: 20.83%
Layer: conv2.weight | Non-zero params: 1667 | Total params: 4608 | Sparsity: 63.82%
Layer: lin2.weight | Non-zero params: 8782 | Total params: 15680 | Sparsity: 43.99%
Model has 10563 non-zero params in total

Approximate model size: 41.26 KB
-------------------------------------
Prune the model with amount = 40.0 % :
-------------------------------------
Evaluating 80% accurate Model
Accuracy on the 10000 test images:  83.16 %

Size of 80% accurate Model
Total parameters: 20432

Parameters per layer:
Layer: conv1.weight | Non-zero params: 114 | Total params: 144 | Sparsity: 20.83%
Layer: conv2.weight | Non-zero params: 1268 | T

In [ ]:
### Combine both quantization and pruning to achieve an 85% accurate model
### that is as small as possible

quantized_model = deepcopy(load_model)
new_sd = load_model.state_dict()

# Quantize the model
n_bits = 4
print(f'Perform per-layer quantization with bit width of {n_bits} bits')
for name,_ in load_model.named_parameters():
    # print('quantizing ',name)
    new_sd[name] = squant(torch.max(torch.abs(sd[name])), n_bits, sd[name])

quantized_model.load_state_dict(new_sd)

prune_amounts = [0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36]

# Run only Channel-wise pruning so we don't completely destroy accuracy
for amount in prune_amounts:
    print(f'Prune the model with amount = {amount * 100} % :')
    pruned_model = deepcopy(quantized_model)
    for name, module in pruned_model.named_modules():
        if isinstance(module, nn.Conv2d):
            # prune.ln_structured(module, name='weight', amount=amount, n=1, dim=0)
            prune.ln_structured(module, name='weight', amount=amount, n=1, dim=1)
            prune.ln_structured(module, name='weight', amount=amount, n=2, dim=1)
            prune.remove(module, 'weight')

    evaluate_model(pruned_model, "85% accurate Model")

Perform per-layer quantization with bit width of 4 bits
Prune the model with amount = 30.0 % :
-------------------------------------
Evaluating 85% accurate Model
Accuracy on the 10000 test images:  85.71 %

Size of 85% accurate Model
Total parameters: 20432

Parameters per layer:
Layer: conv1.weight | Non-zero params: 110 | Total params: 144 | Sparsity: 23.61%
Layer: conv2.weight | Non-zero params: 1663 | Total params: 4608 | Sparsity: 63.91%
Layer: lin2.weight | Non-zero params: 8836 | Total params: 15680 | Sparsity: 43.65%
Model has 10609 non-zero params in total

Approximate model size: 41.44 KB
-------------------------------------
Prune the model with amount = 31.0 % :
-------------------------------------
Evaluating 85% accurate Model
Accuracy on the 10000 test images:  85.71 %

Size of 85% accurate Model
Total parameters: 20432

Parameters per layer:
Layer: conv1.weight | Non-zero params: 110 | Total params: 144 | Sparsity: 23.61%
Layer: conv2.weight | Non-zero params: 1663 | T

In [ ]:
### Combine both quantization and pruning to achieve a 90% accurate model

optimized_model = deepcopy(load_model)
new_sd = load_model.state_dict()

# Quantize the model
n_bits = 8
print(f'Perform per-layer quantization with bit width of {n_bits} bits')
for name,_ in load_model.named_parameters():
    # print('quantizing ',name)
    new_sd[name] = squant(torch.max(torch.abs(sd[name])), n_bits, sd[name])

optimized_model.load_state_dict(new_sd)

prune_amounts = [0.01, 0.05, 0.1, 0.15, 0.2]
# Run only Channel-wise pruning
for amount in prune_amounts:
    print(f'Prune the model with amount = {amount * 100} % :')
    for name, module in optimized_model.named_modules():
        if isinstance(module, nn.Conv2d):
            prune.ln_structured(module, name='weight', amount=amount, n=1, dim=0)
            prune.ln_structured(module, name='weight', amount=amount, n=1, dim=1)
            prune.ln_structured(module, name='weight', amount=amount, n=2, dim=1)
            prune.remove(module, 'weight')

    evaluate_model(optimized_model, "80% accurate Model")

Perform per-layer quantization with bit width of 8 bits
Prune the model with amount = 1.0 % :
-------------------------------------
Evaluating 80% accurate Model
Accuracy on the 10000 test images:  90.24 %

Size of 80% accurate Model
Total parameters: 20432

Parameters per layer:
Layer: conv1.weight | Non-zero params: 141 | Total params: 144 | Sparsity: 2.08%
Layer: conv2.weight | Non-zero params: 4499 | Total params: 4608 | Sparsity: 2.37%
Layer: lin2.weight | Non-zero params: 15127 | Total params: 15680 | Sparsity: 3.53%
Model has 19767 non-zero params in total

Approximate model size: 77.21 KB
-------------------------------------
Prune the model with amount = 5.0 % :
-------------------------------------
Evaluating 80% accurate Model
Accuracy on the 10000 test images:  90.27 %

Size of 80% accurate Model
Total parameters: 20432

Parameters per layer:
Layer: conv1.weight | Non-zero params: 133 | Total params: 144 | Sparsity: 7.64%
Layer: conv2.weight | Non-zero params: 3712 | Total 

In [ ]:
class MyConvNet(nn.Module):
    def __init__(self, args):
        super(MyConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.act1  = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.act2  = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.lin2  = nn.Linear(7*7*32, 10, bias=False)

    def asquant(self, max_value, min_value, num_bits, x):
        # Uniform Asymetric quantizer that
        # quantize x into min_value + sf * qx
        # sf: scaling factor
        # qx: integer in range [0,2^bits-1]
        # note: 2^bits different values can be represented, bits >= 1

        sf = (max_value - min_value) / (2**num_bits - 1)
        qx = stochastic_round((x-min_value)/sf)
        qx = torch.clip(qx,min=0,max=2**(num_bits) - 1)
        dqx = min_value + qx * sf
        return dqx


    def save_weights(self, path):
        sd = self.state_dict()
        quant_dict = self.state_dict()
        for name,_ in self.named_parameters():
            # print('quantizing ',name)
            quant_dict[name] = self.asquant(torch.max(sd[name]), torch.min(sd[name]),
                                                      n_bits, sd[name])
        torch.save(quant_dict, path)

    def forward(self, x):
        x = self.conv1(x)
        x = self.act1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.act2(x)
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = self.lin2(x)
        return x

model = MyConvNet(args)
model = model.cuda()

criterion = nn.CrossEntropyLoss()
criterion=criterion.cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 5e-4, momentum=0.9)